IMPORT


In [61]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import math


from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.api import OLS
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = 2000
pd.options.display.max_rows = 2000


#path = '/Koding_With_Kolesh/challenges/RMB_NOWCAST_APRIL/'
cpi = pd.read_csv('CPI_Historic_Values_Zindi_May_23.csv')
#vehicles = pd.read_csv('Naamsa_Vehicle_Sales.csv')
seed = 3

PIVOTING THE DATA


In [62]:
cpi_pivot = cpi.pivot(index = 'Month', columns = 'Category', values = 'Value').reset_index()#changing from a long format to a wide format,
cpi_pivot['Month'] = pd.to_datetime(cpi_pivot['Month'])
cpi_pivot = cpi_pivot.sort_values("Month").reset_index(drop=True)

ADDING JUNE'S DATA MANUALLY FROM https://www.statssa.gov.za/publications/P0141/P0141June2023.pdf


In [63]:
date_str = '2023-06-30'
date_obj = pd.to_datetime(date_str)
new_row = pd.DataFrame({'Month': [date_obj]})
cpi_pivot = pd.concat([cpi_pivot, new_row]).reset_index(drop=True)
cpi_of_june = [110.9, 104.3, 99.6, 110.4, 118.3, 109.8, 110.8, 107.7, 105.4, 109.6, 105.3, 110.0, 112.3]

for i, col in enumerate(cpi_pivot.columns[1:]):
    cpi_pivot.at[17, col] = cpi_of_june[i]

ADD THE july ROW


In [64]:
date_str = '2023-07-31'
date_obj = pd.to_datetime(date_str)
new_row = pd.DataFrame({'Month': [date_obj]})
cpi_pivot = pd.concat([cpi_pivot, new_row]).reset_index(drop=True)

FEATURE ENGINNERING


In [65]:
feats_to_lag = cpi_pivot.columns[1:].to_list()
for col in feats_to_lag:
    for i in range(1,5):
        cpi_pivot[f'prev_{i}_month_{col}'] = cpi_pivot[col].shift(i)

HANDLE MISSING DATA


In [66]:
cpi_pivot = cpi_pivot.drop(0)
cpi_pivot = cpi_pivot.bfill()

TRAIN AND VALIDATION


In [67]:
train = cpi_pivot[cpi_pivot['Month'] != "2023-07-31"]
test = cpi_pivot[cpi_pivot['Month'] == "2023-07-31"]

training_set = train[train['Month']!= '2023-06-30']
validation_set = train[train['Month']== '2023-06-30']

train.shape, test.shape, training_set.shape, validation_set.shape

((17, 66), (1, 66), (16, 66), (1, 66))

MODELING


In [68]:
import optuna


target_cols = ['Alcoholic beverages and tobacco', 'Clothing and footwear',
       'Communication', 'Education', 'Food and non-alcoholic beverages',
       'Headline_CPI', 'Health', 'Household contents and services',
       'Housing and utilities', 'Miscellaneous goods and services',
       'Recreation and culture', 'Restaurants and hotels ', 'Transport']

#if you add additional data sources that have no value in the predicting month , drop it, now that you have their lags
features= [col for col in train.columns if col not in target_cols + ['Month']]


X_train = training_set[features]
y_train = training_set[target_cols]

X_val = validation_set[features]
y_val = validation_set[target_cols]

l_models = {}
bp = {}

y_predl = []

scaler = MinMaxScaler()

def objective(trial):
    alpha = trial.suggest_loguniform('alpha', 1e-5, 100)  # Ridge regularization strength
    fit_intercept = trial.suggest_categorical('fit_intercept', [True, False])
    copy_X = trial.suggest_categorical('copy_X', [True, False])
    max_iter = trial.suggest_int('max_iter', 100, 1000)
    tol = trial.suggest_loguniform('tol', 1e-6, 1e-3)
    solver = trial.suggest_categorical('solver', ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'])

    r_model = Ridge(alpha=alpha, fit_intercept=fit_intercept, copy_X=copy_X, max_iter=max_iter, tol=tol, solver=solver,)
    r_model.fit(X_train, y_train[target_col])
    y_pred_colr = r_model.predict(X_val)
    return mean_squared_error(y_val[target_col], y_pred_colr)

#training
for target_col in target_cols:
    #study = optuna.create_study(direction='minimize')
    #study.optimize(objective, n_trials=50)
    #best_params = study.best_params
    l_model = Lasso(alpha=0.076, random_state=seed)
    
    X_train_scaled = scaler.fit_transform(X_train)
    l_model.fit(X_train_scaled, y_train[target_col])
    l_models[target_col] = l_model
    #bp[target_col] = best_params
#validation
for target_col in target_cols:
    l_model = l_models[target_col]
    X_val_scaled = scaler.transform(X_val)
    y_pred_coll = l_model.predict(X_val_scaled)
    y_predl.append(y_pred_coll)

y_predl = np.array(y_predl).T

dfl = pd.DataFrame({'y_pred': y_predl.flatten(), 'y_val': y_val.values.flatten()})

rmsel = np.sqrt(mean_squared_error(dfl['y_pred'], dfl['y_val']))

print(f'RMSE of Lasso Regression: {rmsel}') 

RMSE of Lasso Regression: 2.7632279989955912


In [69]:
#best_params

In [70]:
X_train = train[features]
y_train = train[target_cols]

X_val = test[features]
y_val = test[target_cols]


l_models = {}
r_models = {}

y_predl = []


scaler = MinMaxScaler()

#training
for target_col in target_cols:
    l_model = Lasso(alpha=0.076, random_state=seed)
    
    X_train_scaled = scaler.fit_transform(X_train)

    l_model.fit(X_train_scaled, y_train[target_col])
    
    l_models[target_col] = l_model
    
#pridiction
for target_col in target_cols:
    l_model = l_models[target_col]
    
    X_val_scaled = scaler.transform(X_val)
    
    y_pred_coll = l_model.predict(X_val_scaled)

    y_predl.append(y_pred_coll)
    
y_predl = np.array(y_predl).T

print(f'prediction of Lasso Regression: {y_predl}') 

prediction of Lasso Regression: [[110.8478675  104.3625232   99.68823529 110.95038698 118.61014569
  111.58549518 109.862874   108.58698089 107.29591632 110.06886175
  106.17898893 109.8410468  116.22177015]]


SUBMITION


In [71]:
def prepSub(y_pred:list, target_cols: list):
    sub_df = pd.DataFrame(y_pred, columns=target_cols)
    sub_df['Month'] = date_obj
    return sub_df

In [72]:
l = y_predl.tolist()
l[0].insert(0, date_obj)
sub = prepSub(l, ['Month']+target_cols) 

In [73]:
cpi_pivot = cpi.pivot(index = 'Month', columns = 'Category', values = 'Value').reset_index()#changing from a long format to a wide format,
cpi_pivot['Month'] = pd.to_datetime(cpi_pivot['Month'])
cpi_pivot = cpi_pivot.sort_values("Month").reset_index(drop=True)
date_str = '2023-06-30'
date_obj = pd.to_datetime(date_str)
new_row = pd.DataFrame({'Month': [date_obj]})
cpi_pivot = pd.concat([cpi_pivot, new_row]).reset_index(drop=True)
cpi_of_june = [110.9, 104.3, 99.6, 110.4, 118.3, 109.8, 110.8, 107.7, 105.4, 109.6, 105.3, 110.0, 112.3]

for i, col in enumerate(cpi_pivot.columns[1:]):
    cpi_pivot.at[17, col] = cpi_of_june[i]

cpi_pivot = pd.concat([cpi_pivot, sub]).reset_index(drop=True)
date_str = '2023-08-31'
date_obj = pd.to_datetime(date_str)
new_row = pd.DataFrame({'Month': [date_obj]})
cpi_pivot = pd.concat([cpi_pivot, new_row]).reset_index(drop=True)

In [74]:
feats_to_lag = cpi_pivot.columns[1:].to_list()
for col in feats_to_lag:
    for i in range(1,5):
        cpi_pivot[f'prev_{i}_month_{col}'] = cpi_pivot[col].shift(i)

In [75]:
cpi_pivot = cpi_pivot.drop(0)
cpi_pivot = cpi_pivot.bfill()

In [76]:
train = cpi_pivot[cpi_pivot['Month'] != "2023-08-31"]
test = cpi_pivot[cpi_pivot['Month'] == "2023-08-31"]

training_set = train[train['Month']!= '2023-06-30']
validation_set = train[train['Month']== '2023-06-30']

train.shape, test.shape, training_set.shape, validation_set.shape

((18, 66), (1, 66), (17, 66), (1, 66))

In [77]:
import optuna


target_cols = ['Alcoholic beverages and tobacco', 'Clothing and footwear',
       'Communication', 'Education', 'Food and non-alcoholic beverages',
       'Headline_CPI', 'Health', 'Household contents and services',
       'Housing and utilities', 'Miscellaneous goods and services',
       'Recreation and culture', 'Restaurants and hotels ', 'Transport']

#if you add additional data sources that have no value in the predicting month , drop it, now that you have their lags
features= [col for col in train.columns if col not in target_cols + ['Month']]


X_train = training_set[features]
y_train = training_set[target_cols]

X_val = validation_set[features]
y_val = validation_set[target_cols]

x_models = {}
lr_models = {}
l_models = {}
r_models = {}
el_models = {}
bp = {}

y_pred = []
y_predx = []
y_predl = []
y_predr = []
y_predel = []

scaler = MinMaxScaler()

def objective(trial):
    alpha = trial.suggest_loguniform('alpha', 1e-5, 100)  # Ridge regularization strength
    fit_intercept = trial.suggest_categorical('fit_intercept', [True, False])
    copy_X = trial.suggest_categorical('copy_X', [True, False])
    max_iter = trial.suggest_int('max_iter', 100, 1000)
    tol = trial.suggest_loguniform('tol', 1e-6, 1e-3)
    solver = trial.suggest_categorical('solver', ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'])

    r_model = Ridge(alpha=alpha, fit_intercept=fit_intercept, copy_X=copy_X, max_iter=max_iter, tol=tol, solver=solver,)
    r_model.fit(X_train, y_train[target_col])
    y_pred_colr = r_model.predict(X_val)
    return mean_squared_error(y_val[target_col], y_pred_colr)

#training
for target_col in target_cols:
    #study = optuna.create_study(direction='minimize')
    #study.optimize(objective, n_trials=50)
    #best_params = study.best_params
    
    lr_model = LinearRegression()
    x_model = XGBRegressor(seed=seed)
    r_model = Ridge(random_state=seed)
    l_model = Lasso(alpha=0.076, random_state=seed)
    el_model = ElasticNet(alpha=0.1, l1_ratio=0.5, warm_start=True, random_state= seed)
    
    X_train_scaled = scaler.fit_transform(X_train)
    
    lr_model.fit(X_train_scaled, y_train[target_col])
    x_model.fit(X_train_scaled, y_train[target_col])
    r_model.fit(X_train, y_train[target_col])
    l_model.fit(X_train_scaled, y_train[target_col])
    el_model.fit(X_train_scaled, y_train[target_col])
    
    lr_models[target_col] = lr_model
    l_models[target_col] = l_model
    r_models[target_col] = r_model
    el_models[target_col] = el_model
    x_models[target_col] = x_model
    #bp[target_col] = best_params
#validation
for target_col in target_cols:
    lr_model = lr_models[target_col]
    x_model = x_models[target_col]
    l_model = l_models[target_col]
    r_model = r_models[target_col]
    el_model = el_models[target_col]
    
    X_val_scaled = scaler.transform(X_val)
    
    y_pred_col = lr_model.predict(X_val_scaled)
    y_pred_colx = x_model.predict(X_val_scaled)
    y_pred_coll = l_model.predict(X_val_scaled)
    y_pred_colr = r_model.predict(X_val)
    y_pred_colel = el_model.predict(X_val_scaled)
    
    y_pred.append(y_pred_col)
    y_predx.append(y_pred_colx)
    y_predl.append(y_pred_coll)
    y_predr.append(y_pred_colr)
    y_predel.append(y_pred_colel)

#scoring

y_pred = np.array(y_pred).T
y_predx = np.array(y_predx).T
y_predl = np.array(y_predl).T
y_predr = np.array(y_predr).T
y_predel = np.array(y_predel).T

df = pd.DataFrame({'y_pred': y_pred.flatten(), 'y_val': y_val.values.flatten()})
dfx = pd.DataFrame({'y_pred': y_predx.flatten(), 'y_val': y_val.values.flatten()})
dfl = pd.DataFrame({'y_pred': y_predl.flatten(), 'y_val': y_val.values.flatten()})
dfr = pd.DataFrame({'y_pred': y_predr.flatten(), 'y_val': y_val.values.flatten()})
dfel = pd.DataFrame({'y_pred': y_predel.flatten(), 'y_val': y_val.values.flatten()})

#calculate the rmse
rmse = np.sqrt(mean_squared_error(df['y_pred'], df['y_val']))
rmsex = np.sqrt(mean_squared_error(dfx['y_pred'], df['y_val']))
rmsel = np.sqrt(mean_squared_error(dfl['y_pred'], df['y_val']))
rmser = np.sqrt(mean_squared_error(dfr['y_pred'], df['y_val']))
rmseel = np.sqrt(mean_squared_error(dfel['y_pred'], df['y_val']))

print(f'RMSE of Liner Regression: {rmse}') 
print(f'RMSE of XGB: {rmsex}') 
print(f'RMSE of Lasso Regression: {rmsel}') 
print(f'RMSE of Riged Regression: {rmser}') 
print(f'RMSE of Elastic Regression: {rmseel}') 

RMSE of Liner Regression: 1.0704312862123475
RMSE of XGB: 1.0367080360360854
RMSE of Lasso Regression: 1.4450945951585412
RMSE of Riged Regression: 0.8902007108022848
RMSE of Elastic Regression: 1.253532549571102


In [78]:
X_train = train[features]
y_train = train[target_cols]

X_val = test[features]
y_val = test[target_cols]

x_models = {}
lr_models = {}
l_models = {}
r_models = {}
el_models = {}

y_pred = []
y_predx = []
y_predl = []
y_predr = []
y_predel = []

scaler = MinMaxScaler()

#training
for target_col in target_cols:
    lr_model = LinearRegression()
    x_model = XGBRegressor(seed=seed)
    r_model = Ridge(random_state=seed)
    l_model = Lasso(alpha=0.076, random_state=seed)
    el_model = ElasticNet(alpha=0.1, l1_ratio=0.5, warm_start=True, random_state= seed)
    
    X_train_scaled = scaler.fit_transform(X_train)
    
    lr_model.fit(X_train_scaled, y_train[target_col])
    x_model.fit(X_train_scaled, y_train[target_col])
    r_model.fit(X_train, y_train[target_col])
    l_model.fit(X_train_scaled, y_train[target_col])
    el_model.fit(X_train_scaled, y_train[target_col])
    
    lr_models[target_col] = lr_model
    l_models[target_col] = l_model
    r_models[target_col] = r_model
    el_models[target_col] = el_model
    x_models[target_col] = x_model
    
#pridiction
for target_col in target_cols:
    lr_model = lr_models[target_col]
    x_model = x_models[target_col]
    l_model = l_models[target_col]
    r_model = r_models[target_col]
    el_model = el_models[target_col]
    
    X_val_scaled = scaler.transform(X_val)
    
    y_pred_col = lr_model.predict(X_val_scaled)
    y_pred_colx = x_model.predict(X_val_scaled)
    y_pred_coll = l_model.predict(X_val_scaled)
    y_pred_colr = r_model.predict(X_val)
    y_pred_colel = el_model.predict(X_val_scaled)
    
    y_pred.append(y_pred_col)
    y_predx.append(y_pred_colx)
    y_predl.append(y_pred_coll)
    y_predr.append(y_pred_colr)
    y_predel.append(y_pred_colel)
    
y_pred = np.array(y_pred).T
y_predx = np.array(y_predx).T
y_predl = np.array(y_predl).T
y_predr = np.array(y_predr).T
y_predel = np.array(y_predel).T

print(f'pridiction of Liner Regression: {y_pred}') 
print(f'prediction of XGB: {y_predx}') 
print(f'pridiction of Riged Regression: {y_predr}') 
print(f'prediction of Lasso Regression: {y_predl}') 
print(f'pridiction of Elastic Net Regression: {y_predel}') 

pridiction of Liner Regression: [[110.39718281 104.86505082  99.5876797  110.0390104  119.37365244
  112.19309101 110.81057962 108.91518218 107.55440163 110.75957536
  106.30967742 110.94956821 118.50389461]]
prediction of XGB: [[110.88971  104.361244  99.60472  110.94908  118.60893  111.58383
  110.79868  108.58507  107.29439  110.0677   106.17745  109.963715
  113.63125 ]]
pridiction of Riged Regression: [[110.21383066 104.59989864  99.41993697 110.17422624 118.81527343
  111.65873282 109.95766944 108.30199744 107.38828481 109.95140621
  106.27901448 109.93085815 118.00540043]]
prediction of Lasso Regression: [[111.15920661 104.06224871  99.68823529 109.97470639 120.03083875
  111.22070318 110.2281588  108.47004943 107.01191681 109.88880153
  105.29765745 110.45911957 117.35789227]]
pridiction of Elastic Net Regression: [[111.0069696  104.36865396  99.68823529 110.09290243 120.08713764
  111.36268078 109.97100364 108.7410562  106.58047616 109.91805333
  105.71162192 110.42821608 116.

In [79]:
def prepSub(y_pred:list, target_cols: list, test, prefix:str):
    sub_df = pd.DataFrame(y_pred, columns=target_cols)
    sub_df['Month'] = test['Month']
    
    sub_df.set_index('Month', inplace=True)
    sub_df.columns = [prefix+'_' + col.lower().replace('_', ' ').strip() for col in sub_df.columns]
    sub_df.rename(columns= {f"{prefix}_headline cpi": f"{prefix}_headline CPI"}, inplace=True)
    
    sub_df = pd.melt(sub_df.reset_index(), id_vars= ['Month'], var_name= 'ID', value_name= 'Value')
    
    return sub_df[['ID', 'Value']]

In [80]:
sub = prepSub(y_predl, target_cols, test, 'August') 
sub.to_csv('out/lassoa0076_with_lag5_historic.csv', index=False)